In [1]:
import pandas as pd 
import scsims
from sklearn.preprocessing import LabelEncoder
import sys

sys.path.append('../src')
sys.path.append('../tests')
from models.lib.lightning_train import DataModule, generate_trainer
from models.lib.neural import GeneClassifier
from models.lib.data import *

Let's do some simple preprocessing for training, and then test our library

In [3]:
le = LabelEncoder()

labels = pd.read_csv('../data/retina/retina_labels.tsv', sep='\t')
labels = labels[labels['CellType'].isin(['retinal rod cell type B', 'retinal rod cell type A'])]
labels['class_label'] = le.fit_transform(labels['CellType'])

labels.index.name = 'cell'
labels = labels.iloc[0:500, :]
labels.to_csv('../data/retina/retina_labels_numeric.csv')

# label_df = pd.read_csv('../data/retina/retina_labels_numeric.csv', index_col='cell')
# # label_df

# labels.to_csv('../data/retina/retina_labels_numeric.csv')

label_df = pd.read_csv('../data/retina/retina_labels_numeric.csv', index_col='cell')
label_df

,index,Expressed Genes,UMI Count,Percent Mitochond.,BroadCellType,CellType,Donor,class_label
cell,,,,,,,,
0,0024369980fd003553cbc9dfe29f7f95,2351,6060.0,4.125413,retinal rod cell,retinal rod cell type A,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,0
2,0037f1b36684cb59b84d3585ca55ff69,950,1507.0,0.597213,retinal rod cell,retinal rod cell type B,427c0a62-9baf-42ab-a3a3-f48d10544280,1
3,00390952646f52d11a9ab9bba7d6ac51,961,1962.0,7.543323,retinal rod cell,retinal rod cell type A,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,0
4,005b3351658380695a5dc46c384d72d7,858,1384.0,0.144509,retinal rod cell,retinal rod cell type B,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,1
6,00906832f470fc434a52ac7d678a95bc,532,1054.0,6.451613,retinal rod cell,retinal rod cell type A,427c0a62-9baf-42ab-a3a3-f48d10544280,0
...,...,...,...,...,...,...,...,...
838,5591cc519f5883a123893d36bcfb845e,1382,2485.0,0.120724,retinal rod cell,retinal rod cell type B,66b7152c-8e94-412b-b0e4-f8fbf554df5a,1
841,55b15dc75725c3e5e08272e2878a853d,1058,1806.0,0.609081,retinal rod cell,retinal rod cell type B,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,1
842,55bce97854c5e6ac5ee8a97e61c4422f,953,1636.0,0.000000,retinal rod cell,retinal rod cell type B,b8049daa-7458-47bf-8ec2-3f5c56d2cb34,1


In [4]:
datafiles = ['../data/retina/retina_T.csv']
labelfiles = ['../data/retina/retina_labels_numeric.csv']

module = DataModule(
    datafiles=['../data/retina/retina_T.csv'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    batch_size=4,
    num_workers=0,
    skip=3,
    shuffle=True,
    drop_last=True,
    index_col='cell',
)

total_class_weights(labelfiles, 'class_label')

tensor([1.0549, 0.9506])

In [4]:
model = GeneClassifier(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    weights=None
)

Model initialized. input_dim = 37475, output_dim = 2. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = False


In [ ]:
trainer, model, module = generate_trainer(
    datafiles=['../data/retina/retina_T.csv'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    index_col='cell',
    batch_size=4,
    num_workers=0,
    skip=3,
    shuffle=True,
    drop_last=True,
    weighted_metrics=False,
    normalize=True,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.02,
        'weight_decay': 3e-4,
    },
    weights=total_class_weights(labelfiles, 'class_label'),
)

trainer.fit(model, datamodule=module)

Device is cpu
../data/retina/retina_T.csv exists, continuing...
../data/retina/retina_labels_numeric.csv exists, continuing...



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Model initialized. input_dim = 37475, output_dim = 2. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = False
Creating train/val/test DataLoaders...
Done, continuing to training.


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 2.5 M 
----------------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.916     Total estimated model params size (MB)


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [6]:
module.setup()

trainloader = module.trainloader

Creating train/val/test DataLoaders...
Done, continuing to training.


In [7]:
trainloader.dataset[0:50]

[(tensor([0., 1., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 0),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 0),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 0),
 (tensor([0., 1., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 1., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 0),
 (tensor([1., 1., 0.,  ..., 0., 0., 0.]), 0),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 0),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0., 0.]), 1),
 (tensor([0., 0., 0.,  ..., 0., 0.

In [8]:
def _test_first_n_samples(n, datafile, labelfile):
    data = GeneExpressionData(
        datafile, 
        labelfile, 
        'class_label', 
        skip=3,
        index_col='cell'
    )
    cols = data.columns
    
    # Generate dict with half precision values to read this into my 16gb memory
    dtype_cols = dict(zip(cols, [np.float32]*len(cols)))
    
    data_df = pd.read_csv(datafile, nrows=2*n, header=1, dtype=dtype_cols) # Might need some extras since numerical index drops some values
    label_df = pd.read_csv(labelfile, nrows=n)

    similar = []
    for i in range(n):
        datasample = data[i][0]

        dfsample = torch.from_numpy(data_df.loc[label_df.loc[i, 'cell'], :].values).float()
        isclose = all(torch.isclose(datasample, dfsample))
        similar.append(isclose)
    
    print(f"First {n=} columns of expression matrix is equal to GeneExpressionData: {all(p for p in similar)}")

    assert (all(p for p in similar))

_test_first_n_samples(100, '../data/retina/retina_T.csv', '../data/retina/retina_labels_numeric.csv')


First n=100 columns of expression matrix is equal to GeneExpressionData: True


In [9]:
idk = module.trainloader.dataset._labeldf['cell']
idk

0     128
1      82
2      10
3      20
4       6
     ... 
59      3
60      0
61      2
62     91
63    167
Name: cell, Length: 64, dtype: int64

In [10]:
df = pd.read_csv('../data/retina/retina_labels_numeric.csv')
labels = df.loc[:, 'class_label']

In [11]:
df.loc[idk, :]

KeyError: '[128, 137, 116, 113, 125, 108, 151, 168, 124, 144, 165, 142, 138, 121, 159, 146, 100, 155, 147, 140, 103, 158, 134, 135, 106, 153, 120, 167] not in index'

In [ ]:
from sklearn.model_selection import train_test_split 

trainsplit, valsplit = train_test_split(labels, stratify=labels)

trainsplit, valsplit

In [ ]:
df.loc[trainsplit, :]

In [ ]:
df.to_csv('test.csv')

pd.read_csv('test.csv')

In [ ]:
df.to_csv('')